<h1 style="color:#4E2A84;">AWS - Braket - Rigetti - Swap Example</h1>

This notebook provides example of using swap gates to swap qubits' state with idle ancilla qubits. As a potential application of this swap operation, this example shows how to sequentially excute multiple copies of a circuit by inserting the swap gates between each copy of the circuit. The swap operations effectivley reset the qubits since the |0> state of the idle ancilla is swapped into the original qubits, allowing new copy of the circuit to execute on the original qubits.

This notebook leverages the `AWS` cloud service, it uses the `Braket` development environment and targets a `Rigetti` superconducting quantum computer.

In [2]:
# reset all notebook variables just in case
%reset -f

In [3]:
# general imports
import matplotlib.pyplot as plt

# magic word for producing visualizations in notebook
%matplotlib inline
import string

# AWS imports: import Braket SDK modules
from braket.circuits import Circuit, Gate, Observable
from braket.devices import LocalSimulator
from braket.aws import AwsDevice, AwsQuantumTask
from braket.error_mitigation import Debias

In [4]:
# sample circuit
circ = Circuit()

circ.h(0)
circ.cnot(0,1)

# print circuit for visual inspection
print(circ)

T  : │  0  │  1  │
      ┌───┐       
q0 : ─┤ H ├───●───
      └───┘   │   
            ┌─┴─┐ 
q1 : ───────┤ X ├─
            └───┘ 
T  : │  0  │  1  │


In [5]:
# simulate the circuit
device = LocalSimulator()
result = device.run(circ, shots=1000).result()

# show measurement results
counts = result.measurement_counts
print("Measurement counts:", counts)

Measurement counts: Counter({'11': 513, '00': 487})


In [7]:
# set how many qubits does the input circuit uses and index of first qubit
num_qubits = 2
start_index = 0


circ_with_swaps = Circuit()
circ_with_swaps += circ

curr_index = start_index

num_copies = 4

for i in range(num_copies-1): # subtract one because original circuit is the 1st copy, adding num_copies - 1 after that
    for j in range(num_qubits):
        from_qubit = start_index + j
        to_qubit = num_qubits * (i+1) + j + curr_index
        circ_with_swaps.swap(from_qubit,to_qubit)
        
    circ_with_swaps += circ
    print()
        

print(circ_with_swaps)




T  : │  0  │  1  │        2        │  3  │  4  │        5        │  6  │  7  │        8        │  9  │ 10  │
      ┌───┐                         ┌───┐                         ┌───┐                         ┌───┐       
q0 : ─┤ H ├───●──────x──────────────┤ H ├───●──────x──────────────┤ H ├───●──────x──────────────┤ H ├───●───
      └───┘   │      │              └───┘   │      │              └───┘   │      │              └───┘   │   
            ┌─┴─┐    │                    ┌─┴─┐    │                    ┌─┴─┐    │                    ┌─┴─┐ 
q1 : ───────┤ X ├────┼────────x───────────┤ X ├────┼────────x───────────┤ X ├────┼────────x───────────┤ X ├─
            └───┘    │        │           └───┘    │        │           └───┘    │        │           └───┘ 
                     │        │                    │        │                    │        │                 
q2 : ────────────────x────────┼────────────────────┼────────┼────────────────────┼────────┼─────────────────
                

In [8]:
# simulate the circuit
device = LocalSimulator()
result = device.run(circ_with_swaps, shots=1000).result()

# show measurement results
counts = result.measurement_counts
print("Measurement counts:", counts)

Measurement counts: Counter({'00001111': 81, '11000000': 79, '00111111': 75, '00001100': 67, '00110000': 66, '11111100': 65, '11111111': 65, '11110000': 63, '00000000': 59, '00111100': 58, '11110011': 58, '11001100': 56, '11000011': 55, '00110011': 54, '00000011': 53, '11001111': 46})


In [9]:
# import json to help pretty print dictionaries
import json

# helper function to check if nth bit is set in a string representing a binary number
# note bit counting starts at 0, i.e. 0th bit is the least significant bit
def is_nth_bit_set(binary_str, n):
    # Reverse the string to access LSB at index 0
    reversed_str = binary_str[::-1]

    if n >= len(reversed_str):
        return False  # Bit doesn't exist

    return reversed_str[n] == '1'

# helper function to check if bits at specific position match a pattern
def is_bit_pattern_set(binary_str, start, end, pattern_str):
    return binary_str[start:end] == pattern_str

counts_separated = dict()

for i in range(num_copies):
    
    temp_counts = dict()
    
    for j in range(2**num_qubits):
        binary_str = bin(j)[2:].zfill(num_qubits)
        temp_counts[binary_str] = 0
    
    for j in range(2**num_qubits):
        for key in counts.keys():
            binary_str = bin(j)[2:].zfill(num_qubits)
            if is_bit_pattern_set(key, i*num_qubits, (i+1)*num_qubits, binary_str):
            # print(i,j,binary_str)
                temp_counts[binary_str] = temp_counts[binary_str] + counts[key]
    
    counts_separated[i] = temp_counts

print(json.dumps(counts_separated, indent=4))

{
    "0": {
        "00": 513,
        "01": 0,
        "10": 0,
        "11": 487
    },
    "1": {
        "00": 496,
        "01": 0,
        "10": 0,
        "11": 504
    },
    "2": {
        "00": 487,
        "01": 0,
        "10": 0,
        "11": 513
    },
    "3": {
        "00": 513,
        "01": 0,
        "10": 0,
        "11": 487
    }
}


<h1 style="color:#4E2A84;">Estimate qBraids Credits Needed</h1> 

The cost to run a circuit on a quantum computer is divided into per-task cost (effectively the cost to submit a circuit for execution) and per-shot cost (additional cost for each shot).

You can view the cost by selecting $DEVICES$ on the right-hand side of the editor, then search for Rigetti Ankaa-3 (AWS) and finally selecting the pricing cell. <img src="images/image_pricing.png" alt="View pricing" style="width:300px;"/>

The current cost is $30$ qBraid Credits per-task on Ankaa-3 and then $0.09$ qBraid Credits per-task.

Currently each qBraid credit is valued at $0.01$ USD. This means that $100$ credits equate to $1.00$ USD.

In [10]:
# set number of shots
num_shots = 1000

In [11]:
# compute cost
perTaksCost = 30.00
perShotCost = 0.09
totalCost = perTaksCost+perShotCost*num_shots

# print cost
print(f'Cost to run Bell circuit on Rigetti Ankaa-3 via AWS is:')
print(f'{totalCost} qBraid Credit or equivalently ${totalCost/100} USD')

Cost to run Bell circuit on Rigetti Ankaa-3 via AWS is:
120.0 qBraid Credit or equivalently $1.2 USD


<h1 style="color:#4E2A84;">Estimate Cost Saving of Running N Copies</h1>

Check the cost to execute the circuit when multiple circuits are run sequentially, and total number of shots is reduced.

In [12]:
# new number of shots
import math

new_num_shots = math.ceil(num_shots/num_copies)

In [13]:
# print number of copies being used
print('Number of copies:',num_copies)
print()

print('Original number of shots:',num_shots)
print()

print('Reduced number of shots:',new_num_shots)
print()

# compute cost
perTaksCost = 30.00
perShotCost = 0.09
totalCostWithCopies = perTaksCost+perShotCost*new_num_shots

# print cost
print(f'Cost to run Bell circuit on Rigetti Ankaa-3 via AWS is:')
print(f'{totalCostWithCopies} qBraid Credit or equivalently ${totalCostWithCopies/100} USD')
print()

# savings
print(f'Savings ${totalCost/100 - totalCostWithCopies/100} USD or {100*(totalCost/100 - totalCostWithCopies/100)/(totalCost/100)}%')

Number of copies: 4

Original number of shots: 1000

Reduced number of shots: 250

Cost to run Bell circuit on Rigetti Ankaa-3 via AWS is:
52.5 qBraid Credit or equivalently $0.525 USD

Savings $0.6749999999999999 USD or 56.25%


<h1 style="color:#4E2A84;">Submit the Circuit to Rigetti Ankaa-3 and Obtain the Results</h1>

You can now submit the circuit for execution.

In [14]:
# circuit to run
circ_to_run = circ_with_swaps

# print circuit
print(circ_to_run)

T  : │  0  │  1  │        2        │  3  │  4  │        5        │  6  │  7  │        8        │  9  │ 10  │
      ┌───┐                         ┌───┐                         ┌───┐                         ┌───┐       
q0 : ─┤ H ├───●──────x──────────────┤ H ├───●──────x──────────────┤ H ├───●──────x──────────────┤ H ├───●───
      └───┘   │      │              └───┘   │      │              └───┘   │      │              └───┘   │   
            ┌─┴─┐    │                    ┌─┴─┐    │                    ┌─┴─┐    │                    ┌─┴─┐ 
q1 : ───────┤ X ├────┼────────x───────────┤ X ├────┼────────x───────────┤ X ├────┼────────x───────────┤ X ├─
            └───┘    │        │           └───┘    │        │           └───┘    │        │           └───┘ 
                     │        │                    │        │                    │        │                 
q2 : ────────────────x────────┼────────────────────┼────────┼────────────────────┼────────┼─────────────────
                   

In [15]:
# set up device
ankaa = AwsDevice("arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-3")

# run circuit
ankaa_task = ankaa.run(circ_to_run, shots=1000, disable_qubit_rewiring=False)
    
# get id and status of submitted task
ankaa_task_id = ankaa_task.id
ankaa_status = ankaa_task.state()

# print task status
print(f'Status of task: {ankaa_status}\n')

# print task id for future reference
print(f'Task id: {ankaa_task_id}')

Status of task: COMPLETED

Task id: arn:aws:braket:us-west-1:592242689881:quantum-task/af6a3c9d-ba50-431d-b4d8-14e4dde71b79


In [17]:
# get task arn
ankaa_task_arn = 'arn:aws:braket:us-west-1:592242689881:quantum-task/af6a3c9d-ba50-431d-b4d8-14e4dde71b79'

# load the quantum task
ankaa_task_id = AwsQuantumTask(arn=ankaa_task_arn)

# print status
status = ankaa_task_id.state()
print("Status of the task:", status)

print(ankaa_task_id.state())
print(ankaa_task_id.metadata())  # Includes status, failure reason

Status of the task: COMPLETED
COMPLETED
{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'x-powered-by': 'Express', 'access-control-allow-origin': '*', 'access-control-allow-methods': 'GET,PUT,POST,DELETE', 'access-control-allow-headers': 'Content-Type, id-token, refresh-token, email, domain, x-custom-user-agent, api-key', 'access-control-allow-credentials': 'true', 'content-type': 'application/json; charset=utf-8', 'etag': 'W/"45a-qpN8SQJ9nK+DLb+5GONO23HLe04"', 'x-cloud-trace-context': '387a4de7473740016dc7ad93a2d85ba0', 'date': 'Tue, 27 May 2025 18:34:53 GMT', 'server': 'Google Frontend', 'content-length': '1114', 'via': '1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}, 'RetryAttempts': 0}, 'createdAt': datetime.datetime(2025, 5, 27, 18, 34, 15, 479000, tzinfo=tzlocal()), 'deviceArn': 'arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-3', 'deviceParameters': '{"braketSchemaHeader": {"name": "braket.device_schema.rigetti.rigetti_device_parameters

<h1 style="color:#4E2A84;">Analyze Results</h1>



In [19]:
# get task arn
ankaa_task_arn = 'arn:aws:braket:us-west-1:592242689881:quantum-task/af6a3c9d-ba50-431d-b4d8-14e4dde71b79'

# load the quantum task
ankaa_task_id = AwsQuantumTask(arn=ankaa_task_arn)

# get results
ankaa_results = ankaa_task_id.result()

# get measurement counts
ankaa_counts = ankaa_results.measurement_counts
print("Measurement counts:", ankaa_counts)

Measurement counts: Counter({'00001100': 50, '11111100': 50, '00000011': 47, '00111100': 42, '00000000': 40, '00110000': 37, '11000000': 36, '00001111': 35, '11110000': 35, '00111111': 35, '11000011': 34, '11001100': 34, '11111111': 31, '11001111': 31, '00110011': 30, '11110011': 22, '11000111': 13, '11100000': 11, '11100011': 11, '11101111': 11, '11000100': 10, '00110111': 10, '00111011': 10, '00011100': 9, '01001100': 9, '00111000': 9, '00000111': 9, '00011111': 8, '11101100': 8, '11110111': 8, '00101111': 8, '00101100': 7, '00110010': 7, '10111111': 7, '00000010': 6, '11010000': 6, '11010011': 6, '11011111': 6, '00010000': 6, '01001111': 6, '11110100': 6, '00010011': 6, '10010011': 5, '00110100': 5, '00100011': 5, '10111100': 5, '11111011': 5, '00001000': 5, '00111110': 5, '01110011': 4, '11000001': 4, '00100000': 4, '01000011': 4, '11000010': 4, '11100111': 4, '11110001': 4, '01000000': 4, '00111101': 4, '10110000': 3, '01100011': 3, '00010111': 3, '01111100': 3, '11101011': 3, '00

In [20]:
# import json to help pretty print dictionaries
import json

# helper function to check if nth bit is set in a string representing a binary number
# note bit counting starts at 0, i.e. 0th bit is the least significant bit
def is_nth_bit_set(binary_str, n):
    # Reverse the string to access LSB at index 0
    reversed_str = binary_str[::-1]

    if n >= len(reversed_str):
        return False  # Bit doesn't exist

    return reversed_str[n] == '1'

# helper function to check if bits at specific position match a pattern
def is_bit_pattern_set(binary_str, start, end, pattern_str):
    return binary_str[start:end] == pattern_str

counts_separated_ankaa = dict()

for i in range(num_copies):
    
    temp_counts = dict()
    
    for j in range(2**num_qubits):
        binary_str = bin(j)[2:].zfill(num_qubits)
        temp_counts[binary_str] = 0
    
    for j in range(2**num_qubits):
        for key in ankaa_counts.keys():
            binary_str = bin(j)[2:].zfill(num_qubits)
            if is_bit_pattern_set(key, i*num_qubits, (i+1)*num_qubits, binary_str):
            # print(i,j,binary_str)
                temp_counts[binary_str] = temp_counts[binary_str] + ankaa_counts[key]
    
    counts_separated_ankaa[i] = temp_counts

print(json.dumps(counts_separated_ankaa, indent=4))

{
    "0": {
        "00": 471,
        "01": 51,
        "10": 46,
        "11": 432
    },
    "1": {
        "00": 421,
        "01": 72,
        "10": 98,
        "11": 409
    },
    "2": {
        "00": 408,
        "01": 98,
        "10": 58,
        "11": 436
    },
    "3": {
        "00": 467,
        "01": 40,
        "10": 45,
        "11": 448
    }
}
